In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from tqdm.notebook import tqdm


In [3]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [5]:
!pip install optuna

     |████████████████████████████████| 308 kB 25.4 MB/s 
     |████████████████████████████████| 210 kB 34.4 MB/s 
     |████████████████████████████████| 81 kB 10.9 MB/s 
     |████████████████████████████████| 78 kB 8.0 MB/s 
     |████████████████████████████████| 113 kB 44.6 MB/s 
     |████████████████████████████████| 49 kB 7.3 MB/s 
     |████████████████████████████████| 146 kB 46.4 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=d5060f2915f7044dad2d82bad6a35de7803b7b60784e42be5bcca1c1ce54ec82
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [6]:
import optuna

In [7]:
train = pd.read_csv('/content/drive/MyDrive/project9/train_transformed.csv')
train_labels = pd.read_csv('/content/drive/MyDrive/project9/music30s_trainlabel.csv')
test =pd.read_csv('/content/drive/MyDrive/project9/test_transformed.csv')


In [8]:
train_labels =train_labels.iloc[:,1:]
train_labels.head(3)


,label
0,reggae
1,reggae
2,country


In [9]:
cols = [c for c in train.columns]

In [10]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
train[cols] = stdsc.fit_transform(train[cols])
test[cols] = stdsc.transform(test[cols])

In [11]:
display(train.head(3))
display(test.head(3))

,length,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_bandwidth_mean,rolloff_mean,zero_crossing_rate_mean,harmony_mean,perceptr_mean,tempo,...,mfcc11_var,mfcc12_var,mfcc13_var,mfcc14_var,mfcc15_var,mfcc16_var,mfcc17_var,mfcc18_var,mfcc19_var,mfcc20_var
0,-0.298576,0.642278,0.177059,1.233038,1.386263,1.266091,0.764469,0.211645,0.314663,-0.234117,...,1.838597,0.833696,1.034494,0.776698,0.990850,1.481935,0.680471,0.666631,0.743034,-0.091657
1,-0.298576,0.227633,-0.595372,-0.050038,-0.362538,-0.174821,0.380594,-0.925636,-1.493513,0.877118,...,0.824710,-0.080086,0.785104,0.768889,1.027054,0.374397,0.127377,0.540703,-0.063098,0.060347
2,-0.142344,-0.050788,-0.477223,0.161218,0.571295,-0.000670,0.009428,0.213437,0.450461,-0.695707,...,0.631436,0.303991,-0.387986,0.346198,0.130285,-0.055250,-0.151197,-0.366009,-1.045659,-1.167858


,length,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_bandwidth_mean,rolloff_mean,zero_crossing_rate_mean,harmony_mean,perceptr_mean,tempo,...,mfcc11_var,mfcc12_var,mfcc13_var,mfcc14_var,mfcc15_var,mfcc16_var,mfcc17_var,mfcc18_var,mfcc19_var,mfcc20_var
0,-0.298576,-0.648987,-1.134700,-1.172775,-0.967173,-1.282262,-0.778708,-1.049366,-0.701822,0.365950,...,1.438900,1.531189,1.933873,1.593768,1.864126,0.803889,1.817925,1.818024,1.261632,1.923743
1,-0.142344,0.703835,-1.919805,-1.723546,-1.199076,-1.699029,-1.720368,0.207379,0.356319,0.877118,...,-1.153414,-1.881063,-3.674341,-3.609437,-3.750683,-3.791344,-3.700374,-3.402824,-2.418003,-1.934408
2,-0.298576,0.939176,2.349185,0.841052,0.373789,0.601497,-0.148723,0.212788,0.200619,-0.948482,...,1.184065,0.593270,1.220743,0.620820,0.712082,0.518463,0.468916,0.123271,-0.013371,0.258653


In [12]:
train = pd.concat([train,train_labels],axis=1)

In [13]:
N_SPLITS = 5
N_REPEATS = 3
EARLY_STOPPING_ROUNDS = 300
VERBOSE = False
SEED = 17171

In [14]:
target_column = 'label'

In [15]:
features = [c for c in train.columns if c not in [target_column]]

In [16]:
X = train.drop('label',axis=1)
y=train['label']

In [20]:
## going by the optuna way:
import optuna
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
# Which hyperparameters to tune: https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

def objective(trial):

    # -- Tune estimator algorithm
    #suggest_int(name, low, high[, step, log])
    C = trial.suggest_float("C", 0,1)
    #algorithm{‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’}, default=’auto’
    penalty = trial.suggest_categorical("penalty",['none', 'l2'])
    logreg = LogisticRegression(multi_class='multinomial',solver='lbfgs',C=C, penalty=penalty)
        
    # -- Cross-validate the features reduced by dimensionality reduction methods
    rskfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)
    score = cross_val_score(logreg, X,y, scoring='accuracy', cv=rskfold, n_jobs = -1, error_score = 'raise')
    score = score.mean()
    return score


In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50, timeout = 4*60*60)

[I 2022-04-05 14:40:57,417] A new study created in memory with name: no-name-c2accf3b-43fa-4491-a5d0-611a5d136ebd
[I 2022-04-05 14:40:58,639] Trial 0 finished with value: 0.6618518518518518 and parameters: {'C': 0.578515417508163, 'penalty': 'none'}. Best is trial 0 with value: 0.6618518518518518.
[I 2022-04-05 14:41:00,014] Trial 1 finished with value: 0.6574074074074076 and parameters: {'C': 0.9482770611495656, 'penalty': 'none'}. Best is trial 0 with value: 0.6618518518518518.
[I 2022-04-05 14:41:01,297] Trial 2 finished with value: 0.7203703703703704 and parameters: {'C': 0.6256409552010144, 'penalty': 'l2'}. Best is trial 2 with value: 0.7203703703703704.
[I 2022-04-05 14:41:02,694] Trial 3 finished with value: 0.7222222222222222 and parameters: {'C': 0.6482940170438026, 'penalty': 'l2'}. Best is trial 3 with value: 0.7222222222222222.
[I 2022-04-05 14:41:03,993] Trial 4 finished with value: 0.7222222222222224 and parameters: {'C': 0.87033331443294, 'penalty': 'l2'}. Best is trial

In [ ]:
trial = study.best_trial    
print('Accuracy: {}'.format(trial.value))   

Accuracy: 0.7381481481481481


In [ ]:
print("Best hyperparameters: {}".format(trial.params))

Best hyperparameters: {'C': 0.4812604306766038, 'penalty': 'l2'}


In [ ]:
optuna.visualization.plot_optimization_history(study)


In [ ]:
logreg_params = trial.params

In [17]:
logreg_params ={'C': 0.4812604306766038, 'penalty': 'l2'}

In [18]:

from sklearn.metrics import recall_score,precision_score,f1_score
from sklearn.metrics import classification_report
import statistics
from statistics import mean

In [24]:
%%time
N_SPLITS = 5
y_preds = []
acc = []
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits = N_SPLITS)

for fold, (train_id, valid_id) in enumerate(folds.split(X,y)):
  X_train = X.iloc[train_id]
  y_train = y.iloc[train_id]
  X_valid = X.iloc[valid_id]
  y_valid = y.iloc[valid_id]
  #0.741111
  model = LogisticRegression(C=0.37555358043161324, penalty='l2',multi_class='multinomial',solver='lbfgs',max_iter =500)
  model.fit(X_train, y_train)
  valid_pred = model.predict(X_valid)

  #print('fold:{}'.format(fold+1))
  ##print('Validation Accuracy score : {}'.format(accuracy_score(y_valid, valid_pred)))
  #print('-'*100)

  acc.append(accuracy_score(y_valid, valid_pred))

  y_preds.append(model.predict(test))
print('the mean accuracy is: {}'.format(mean(acc)))




the mean accuracy is: 0.7266666666666667
CPU times: user 1.04 s, sys: 696 ms, total: 1.73 s
Wall time: 896 ms


In [23]:
y_preds

[array(['reggae', 'reggae', 'hiphop', 'reggae', 'blues', 'country', 'pop',
        'jazz', 'blues', 'blues', 'disco', 'disco', 'metal', 'reggae',
        'jazz', 'country', 'country', 'reggae', 'jazz', 'blues', 'country',
        'hiphop', 'pop', 'classical', 'disco', 'reggae', 'disco', 'metal',
        'pop', 'jazz', 'jazz', 'classical', 'rock', 'classical', 'rock',
        'reggae', 'classical', 'classical', 'pop', 'blues', 'rock',
        'blues', 'jazz', 'classical', 'hiphop', 'hiphop', 'jazz', 'metal',
        'pop', 'pop', 'disco', 'blues', 'pop', 'metal', 'jazz', 'pop',
        'blues', 'jazz', 'disco', 'reggae', 'country', 'pop', 'pop',
        'country', 'classical', 'rock', 'jazz', 'reggae', 'reggae',
        'metal', 'reggae', 'pop', 'metal', 'classical', 'classical',
        'jazz', 'hiphop', 'blues', 'disco', 'rock', 'rock', 'country',
        'reggae', 'blues', 'classical', 'blues', 'jazz', 'disco', 'disco',
        'metal', 'country', 'disco', 'blues', 'metal', 'blues', 

In [ ]:
from collections import Counter
def mode(data):
    """Return the most common data point from discrete or nominal data.
    ``mode`` assumes discrete data, and returns a single value. This is the
    standard treatment of the mode as commonly taught in schools:
        >>> mode([1, 1, 2, 3, 3, 3, 3, 4])
        3
    This also works with nominal (non-numeric) data:
        >>> mode(["red", "blue", "blue", "red", "green", "red", "red"])
        'red'
    If there are multiple modes with same frequency, return the first one
    encountered:
        >>> mode(['red', 'red', 'green', 'blue', 'blue'])
        'red'
    If *data* is empty, ``mode``, raises StatisticsError.
    """
    pairs = Counter(iter(data)).most_common(1)
    try:
        return pairs[0][0]
    except IndexError:
        raise StatisticsError('no mode for empty data') from None




In [ ]:
y_pred = pd.DataFrame(columns=['logreg_label'])
pred= []
for j in range(0,100):
    arr = [y_preds[0][j], y_preds[1][j], y_preds[2][j], y_preds[3][j],y_preds[4][j]]
    pred.append(mode(arr))

In [ ]:
y_pred["logreg_label"]=pred

In [ ]:
y_pred

,logreg_label
0,reggae
1,classical
2,hiphop
3,reggae
4,blues
...,...
95,reggae
96,disco
97,pop
98,hiphop


In [ ]:
y_pred.to_csv('logreg_optuna_pred.csv', index=False)

In [ ]:
!cp -r '/content/logreg_optuna_pred.csv' /content/drive/MyDrive/project9
